In [1]:
seed = 3906303

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.metrics import RootMeanSquaredError
from sklearn.metrics import r2_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, label_binarize, LabelEncoder

import pickle
import glob, os
import gzip
import random
np.random.seed(seed)
random.seed(seed)

## Load dataset

Here we read in all the measurement files created in the datagen notebook. Each dataframe is a separate file, which is each a separate table of measurement readings of particular types.

In [3]:
# Create dictionary of measurement-type dataframes
MEAS = {
    name: pd.read_csv(
        fp,
        parse_dates=['charttime'],       # parsed dates on the fly
        low_memory=False                 # avoids mixed-type warnings
    )
    for fp  in sorted(glob.glob('Datagen/*_filtered_chartevents.csv'))
    for name in [os.path.basename(fp).split('_')[0]]
}

FEATURES = list(MEAS.keys())
F        = len(FEATURES)             # number of features
GLU_IDX  = FEATURES.index('bg')     # which column contains glucose

In [4]:
FEATURES

['abpm', 'bg', 'cvp', 'f', 'hr', 'hrh', 'hrl', 'nbpm', 'rr', 'spo2']

In [5]:
hr = MEAS['hr'].copy()

# Data filtering and cleaning

Below we filter impossible values for the physiological features. No negative numbers, except for rare cases of CVP feature.

In [6]:
#  Per-variable rules. Only reject impossible numbers
LOGIC_RULES = {
    'bg' : lambda s: (s >= 10) & (s <= 2656),                     # historically recorded extreme mg/dL
    'hr'  : lambda s: (s >= 20) & (s <= 600),                     # beats / min
    'rr'  : lambda s: (s >= 6) & (s <= 60),                       # breaths / min
    'spo2': lambda s: (s >= 50) & (s <= 100),                     # % saturation
    'nbpm': lambda s: (s >= 50) & (s <= 130),                     # mm Hg
    'abpm': lambda s: (s >= 50) & (s <= 130),                     # mm Hg
    'f'   : lambda s: (s >= 53) & (s <= 116),                     # F < 0 impossible
    # CVP can dip a few mm Hg negative during inspiration
    'cvp' : lambda s: (s >= -5) & (s <= 20),                      # allow mild physio negative
    'hrl' : lambda s: (s >= 30) & (s <= 100),
    'hrh' : lambda s: (s >= 100) & (s <= 600),
}

def filter_impossible(df, short_name):
    df = df.copy()
    rule = LOGIC_RULES.get(short_name)
    if rule is None:
        raise KeyError(f"No logic rule defined for '{short_name}'")
    return df[rule(df['value'])]

# Apply to every dataframe in the MEAS dict
MEAS_CLEAN = {
    name: filter_impossible(df, name)
    for name, df in MEAS.items()
}

# Invert logic rules to capture invalid (impossible) values
def get_invalid(df, short_name):
    df = df.copy()
    rule = LOGIC_RULES.get(short_name)
    if rule is None:
        raise KeyError(f"No logic rule defined for '{short_name}'")
    return df[~rule(df['value'])]  # Invert the logic to get invalids

# Store invalid rows per measurement
MEAS_INVALID = {
    name: get_invalid(df, name)
    for name, df in MEAS.items()
}

# Print counts of invalid entries
for name, df in MEAS_INVALID.items():
    print(f"{name}: {len(df)} invalid rows")


abpm: 61408 invalid rows
bg: 351 invalid rows
cvp: 73743 invalid rows
f: 1052 invalid rows
hr: 3926 invalid rows
hrh: 5461 invalid rows
hrl: 3986 invalid rows
nbpm: 98189 invalid rows
rr: 43720 invalid rows
spo2: 3251 invalid rows


## Valid data range sources:
- Blood Glucose: 10-2656 mg/dL https://casereports.bmj.com/content/15/7/e245890 (min recorded)
- Heartrate: 20–600 bpm https://en.wikipedia.org/wiki/Heart_rate (max recorded)
- Respiratory Rate: 6–60 breaths per minute https://en.wikipedia.org/wiki/Respiratory_rate
- Oxygen Saturation: 50–100% https://www.verywellhealth.com/blood-oxygen-level-8656297
- Non-Invasive Blood Pressure Mean: 50–130 mmHg https://www.healthline.com/health/mean-arterial-pressure#high-map
- Arterial Blood Pressure Mean: 50–130 mmHg https://www.healthline.com/health/mean-arterial-pressure#high-map
- Temperature Fahrenheit: 53–116°F https://en.wikipedia.org/wiki/Human_body_temperature (lowest recorded)
- Central Venous Pressure: -5 to 20 mmHg (lost source)
- Heart Rate Alarm Low: 30–100 bpm https://www.verywellhealth.com/dangerous-heart-rate-5215509
- Heart Rate Alarm High: 100–600 bpm https://en.wikipedia.org/wiki/Heart_rate (same as above)

# Split Dataset into Training, Validation and Testing Datasets

In [7]:
unique_subjects = MEAS_CLEAN['bg']['subject_id'].unique()
train_subjects, test_subjects = train_test_split(unique_subjects, test_size=0.2,
                                         random_state=seed)
train_subjects, val_subjects  = train_test_split(train_subjects, test_size=0.25,
                                         random_state=seed)

SPLIT = {'train': train_subjects, 'val': val_subjects, 'test': test_subjects}

### Hyperparameters

In [8]:
FREQ       = '2H'           # 2-hour bins
MAX_HOURS  = 48
STEPS      = MAX_HOURS // 2 # 48 h ÷ 2 h = 24 bins
L          = 6               # look-back  steps (12 hours of history)
H          = 1               # look-ahead steps (direct subsequent prediction)
SENTINEL   = -999.0            # value the Masking layer will ignore

### Here we resample the data to ensure there is a row for every timestep interval of 2 hours.

In [9]:
def resample_stay_multi(stay_id, dfs_by_feat):

    mat = np.full((STEPS, F), np.nan, dtype='float32')

    # Find earliest timestamp amongst all features
    start_time = None
    # Visit all feature dataframes one by one, and keep track of earliest recorded time
    for df in dfs_by_feat.values():
        s = df.loc[df['stay_id'] == stay_id, 'charttime']
        if not s.empty:
            t0 = s.min()
            start_time = t0 if start_time is None else min(start_time, t0)

    # Return None if none of the feature dfs have the stay_id
    if start_time is None:
        return None

    # Standardise horizon length of icu stay measurements
    horizon = pd.date_range(start=start_time.floor(FREQ),
                            periods=STEPS,
                            freq=FREQ)

    # Fill one column of matrix per feature
    for j, feat in enumerate(FEATURES):
        g = dfs_by_feat[feat]
        g = g[g['stay_id'] == stay_id]
        if g.empty:
          # Column remains NaNs
            continue
        s = (g.set_index('charttime')['value']
               .resample(FREQ).mean())
        # reindex sets length of values to be exactly STEPS length long,
        # by trancating or padding with NaNs
        mat[:, j] = s.reindex(horizon).values

    return mat                       # (STEPS, F)

Here we create the data to be fed into the model. Each row of data in the X datasets is a sequence of BG measurements. The Y datasets contain a BG reading that follows directly from that sequence after.

The approach we took is so that a sequence of measurements (starting from the first) is created of certain length from each ICU admission, which is taken as the X, and the following value after the sequence is taken as the Y. Then the window is slid over by one so that the sequence is slightly different, creating a new X value and a new following Y value. This process is repeated until the end of the ICU admission measurement sequence is reached, then a new ICU admission is chosen, until all ICU admissions have had their sequences recorded.

In [10]:
def make_Xy_multi(subj_ids, fit_scaler=False, scaler_dict=None):
    X_stays = []

    # Create dictionary of all feature dataframes for the current split
    dfs_by_feat = {}
    for feat, df in MEAS_CLEAN.items():
        df_ = df[df['subject_id'].isin(subj_ids)].copy()
        df_['charttime'] = pd.to_datetime(df_['charttime'])
        dfs_by_feat[feat] = df_

    # Retrieve all unique stay_ids across all feature dataframes
    all_stays = pd.concat([d[['stay_id']] for d in dfs_by_feat.values()]
                          ).drop_duplicates()['stay_id'].values
    # Creates a list of (STEPS, F) matrices
    for sid in all_stays:
        M = resample_stay_multi(sid, dfs_by_feat)   # (STEPS, F) or None
        # Only append the matrix if at least one of the dataframes contains a value
        if M is not None:
            X_stays.append(M)

    # Convert to list of stays to 3D tensor shape
    X_stays = np.stack(X_stays)          # (N_stays, STEPS, F)

    # Feature-wise Standardisation
    if scaler_dict is None:
        scaler_dict = {}
    for j in range(F):
        # Concatenate all feature j columns across all stays, include every row, stack them
        col = X_stays[:, :, j].ravel()
        # NaN mask
        mask = ~np.isnan(col)
        if fit_scaler:
            mu = np.nanmean(col)
            sd = np.nanstd(col)
            # Protects program from division-by-zero error by ensuring if col is all NaNs, 0 is replaced with 1
            sd = sd if sd != 0 else 1.0
            scaler_dict[j] = (mu, sd)
        # Retrieves values for std and mean for when non-training splits are scaled
        mu, sd = scaler_dict[j]
        # Scales non-NaN data
        col[mask] = (col[mask] - mu) / sd

        # Sets a slice of dimension (X_stays.shape[0], STEPS) of a particular feature to the col variable
        X_stays[:, :, j] = col.reshape(X_stays.shape[0], STEPS)

    # Sliding window
    windows, targets = [], []
    for stay in X_stays:
        # Window slides across every point so that the entire horizon of the ICU stay is included once
        for t0 in range(0, STEPS - L - H + 1):
            x_win = stay[t0 : t0 + L, :]                # Look-back window
            y_val = stay[t0 + L + H - 1, GLU_IDX]       # Glucose value following window
            if not np.isnan(y_val):                     # ensure that the final y value is an actual value
                windows.append(x_win)
                targets.append(y_val)

    X = np.array(windows, dtype='float32')             # (N, L, F)
    y = np.array(targets, dtype='float32')             # (N,)

    # mask NaN timesteps
    X[np.isnan(X)] = SENTINEL
    return X, y, scaler_dict

In [11]:
os.makedirs('Pickles', exist_ok=True)

In [12]:
X_train, y_train, SCALERS = make_Xy_multi(SPLIT['train'], fit_scaler=True)

# Save train data and scalers immediately
train_path = 'Pickles/filtered_glu_train.pkl.gz'
with gzip.open(train_path, 'wb') as f:
    pickle.dump({
        'X_train': X_train,
        'y_train': y_train,
        'scalers': SCALERS
    }, f, protocol=pickle.HIGHEST_PROTOCOL)

print(f"Saved train set -> {os.path.getsize(train_path)/1e6:.1f} MB")

Saved train set -> 10.1 MB


In [13]:
X_val, y_val, _ = make_Xy_multi(SPLIT['val'], scaler_dict=SCALERS)

# Save val data immediately
val_path = 'Pickles/filtered_glu_val.pkl.gz'
with gzip.open(val_path, 'wb') as f:
    pickle.dump({
        'X_val': X_val,
        'y_val': y_val
    }, f, protocol=pickle.HIGHEST_PROTOCOL)

print(f"Saved validation set -> {os.path.getsize(val_path)/1e6:.1f} MB")


Saved validation set -> 3.4 MB


In [14]:
X_test, y_test, _ = make_Xy_multi(SPLIT['test'], scaler_dict=SCALERS)

# Save test data immediately
test_path = 'Pickles/filtered_glu_test.pkl.gz'
with gzip.open(test_path, 'wb') as f:
    pickle.dump({
        'X_test': X_test,
        'y_test': y_test
    }, f, protocol=pickle.HIGHEST_PROTOCOL)

print(f"Saved test set -> {os.path.getsize(test_path)/1e6:.1f} MB")

Saved test set -> 3.4 MB


In [15]:
# After you build SCALERS in make_Xy_multi
mu_glu, std_glu = SCALERS[GLU_IDX]        
print(f"Glucose  μ={mu_glu:.1f}  σ={std_glu:.1f}")


Glucose  μ=147.7  σ=64.0


In [16]:
print("Train:", X_train.shape, y_train.shape)
print("Val  :", X_val.shape,   y_val.shape)
print("Test :", X_test.shape,  y_test.shape)

Train: (255142, 6, 10) (255142,)
Val  : (85896, 6, 10) (85896,)
Test : (84763, 6, 10) (84763,)


In [17]:
print("Any NaN in y_train ?", np.isnan(y_train).any())
print("Any NaN in X_train ?", np.isnan(X_train).any())

# How many?
print("NaN count:", np.isnan(X_train).sum())

Any NaN in y_train ? False
Any NaN in X_train ? False
NaN count: 0


In [6]:
BATCH = 64
def make_ds(X, y, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle:
        ds = ds.shuffle(len(y), seed=seed)
    return ds.batch(BATCH).prefetch(tf.data.AUTOTUNE)

In [7]:
ds_train = make_ds(X_train, y_train, shuffle=True)
ds_val   = make_ds(X_val,   y_val,   shuffle=False)
ds_test  = make_ds(X_test,  y_test,  shuffle=False)

In [20]:
model = tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=SENTINEL, input_shape=(L, F)),
    tf.keras.layers.LSTM(64),
    # A dense layer here is necessary, as each hidden state output, which is
    # the output of an LSTM unit is only within the range of a tanh activation function,
    # so further transformation is needed
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)                   # regression
])

model.compile(loss='mse', optimizer='adam', metrics=['mae', RootMeanSquaredError(name='rmse'), 'mse'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 6, 10)             0         
                                                                 
 lstm (LSTM)                 (None, 64)                19200     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 21,313
Trainable params: 21,313
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Create directory if it doesn't exist
os.makedirs('models', exist_ok=True)

In [22]:
#  Create a checkpoint callback
ckpt = tf.keras.callbacks.ModelCheckpoint(
    filepath='models/correctly_filtered_multivar_epoch_{epoch:02d}.h5',
    save_freq='epoch',                        # save after each epoch
    save_weights_only=False,                  # save the entire model (architecture + weights + optimizer state)
    verbose=1)

history = model.fit(
    ds_train,
    epochs=100,
    # Validation is used to determine if model is overfitting
    validation_data=ds_val,
    callbacks=[ckpt],
    verbose=1
)

results = model.evaluate(ds_test, return_dict=True)
print(f"Test RMSE = {results['rmse']:.2f}   |   MAE = {results['mae']:.2f}")

Epoch 1/100
3986/3987 [============================>.] - ETA: 0s - loss: 0.7864 - mae: 0.6199 - rmse: 0.8868 - mse: 0.7864
Epoch 1: saving model to models\correctly_filtered_multivar_epoch_01.h5
3987/3987 [==============================] - 18s 4ms/step - loss: 0.7864 - mae: 0.6199 - rmse: 0.8868 - mse: 0.7864 - val_loss: 0.8048 - val_mae: 0.6173 - val_rmse: 0.8971 - val_mse: 0.8048
Epoch 2/100
3986/3987 [============================>.] - ETA: 0s - loss: 0.7834 - mae: 0.6181 - rmse: 0.8851 - mse: 0.7834
Epoch 2: saving model to models\correctly_filtered_multivar_epoch_02.h5
3987/3987 [==============================] - 16s 4ms/step - loss: 0.7834 - mae: 0.6181 - rmse: 0.8851 - mse: 0.7834 - val_loss: 0.8088 - val_mae: 0.6348 - val_rmse: 0.8993 - val_mse: 0.8088
Epoch 3/100
3987/3987 [==============================] - ETA: 0s - loss: 0.7796 - mae: 0.6159 - rmse: 0.8829 - mse: 0.7796
Epoch 3: saving model to models\correctly_filtered_multivar_epoch_03.h5
3987/3987 [========================

3984/3987 [============================>.] - ETA: 0s - loss: 0.7624 - mae: 0.6076 - rmse: 0.8732 - mse: 0.7624
Epoch 43: saving model to models\correctly_filtered_multivar_epoch_43.h5
3987/3987 [==============================] - 38s 9ms/step - loss: 0.7627 - mae: 0.6076 - rmse: 0.8733 - mse: 0.7627 - val_loss: 0.7892 - val_mae: 0.6079 - val_rmse: 0.8884 - val_mse: 0.7892
Epoch 44/100
3986/3987 [============================>.] - ETA: 0s - loss: 0.7625 - mae: 0.6075 - rmse: 0.8732 - mse: 0.7625
Epoch 44: saving model to models\correctly_filtered_multivar_epoch_44.h5
3987/3987 [==============================] - 42s 10ms/step - loss: 0.7625 - mae: 0.6075 - rmse: 0.8732 - mse: 0.7625 - val_loss: 0.7893 - val_mae: 0.6168 - val_rmse: 0.8884 - val_mse: 0.7893
Epoch 45/100
3986/3987 [============================>.] - ETA: 0s - loss: 0.7632 - mae: 0.6077 - rmse: 0.8736 - mse: 0.7632
Epoch 45: saving model to models\correctly_filtered_multivar_epoch_45.h5
3987/3987 [==============================

3983/3987 [============================>.] - ETA: 0s - loss: 0.7575 - mae: 0.6044 - rmse: 0.8703 - mse: 0.7575
Epoch 85: saving model to models\correctly_filtered_multivar_epoch_85.h5
3987/3987 [==============================] - 55s 14ms/step - loss: 0.7575 - mae: 0.6044 - rmse: 0.8703 - mse: 0.7575 - val_loss: 0.7837 - val_mae: 0.6183 - val_rmse: 0.8853 - val_mse: 0.7837
Epoch 86/100
3985/3987 [============================>.] - ETA: 0s - loss: 0.7581 - mae: 0.6046 - rmse: 0.8707 - mse: 0.7581
Epoch 86: saving model to models\correctly_filtered_multivar_epoch_86.h5
3987/3987 [==============================] - 53s 13ms/step - loss: 0.7580 - mae: 0.6046 - rmse: 0.8706 - mse: 0.7580 - val_loss: 0.7822 - val_mae: 0.6123 - val_rmse: 0.8844 - val_mse: 0.7822
Epoch 87/100
3985/3987 [============================>.] - ETA: 0s - loss: 0.7572 - mae: 0.6045 - rmse: 0.8702 - mse: 0.7572
Epoch 87: saving model to models\correctly_filtered_multivar_epoch_87.h5
3987/3987 [=============================

In [23]:
y_pred_z = model.predict(ds_test, verbose=0).ravel()   # z-scores
y_true_z = y_test

# inverse-transform back to mg/dL
y_pred = y_pred_z * std_glu + mu_glu
y_true = y_true_z * std_glu + mu_glu

# metrics in true units
mse  = np.mean((y_true - y_pred) ** 2)    
rmse = np.sqrt(mse)
mae  = np.mean(np.abs(y_true - y_pred))

print(f"Test MSE  = {mse :.2f} (mg/dL)^2")
print(f"Test RMSE = {rmse:.2f} mg/dL")
print(f"Test MAE  = {mae :.2f} mg/dL")

Test MSE  = 3303.84 (mg/dL)^2
Test RMSE = 57.48 mg/dL
Test MAE  = 39.08 mg/dL


In [25]:
r2 = r2_score(y_true, y_pred)
print(f"Test R^2 = {r2:.4f}")

Test R^2 = 0.0670


Below we load models and data to do further experimentation on the results

In [3]:
model = tf.keras.models.load_model('models/correctly_filtered_multivar_epoch_100.h5')

In [4]:
test_path = 'Pickles/filtered_glu_test.pkl.gz'

with gzip.open(test_path, 'rb') as f:
    test_bundle = pickle.load(f)

X_test = test_bundle['X_test']
y_test = test_bundle['y_test']

In [9]:
train_path = 'Pickles/filtered_glu_train.pkl.gz'

with gzip.open(train_path, 'rb') as f:
    train_bundle = pickle.load(f)

SCALERS = train_bundle['scalers']

# Classification Metrics

Below we convert to classification metrics so we can compare to other models.

In [12]:
y_pred_z = model.predict(ds_test, verbose=0).ravel()   # z-scores
y_true_z = y_test
mu_glu, std_glu = SCALERS[1]          # index 1 is BG scale values

# inverse-transform back to mg/dL
y_pred = y_pred_z * std_glu + mu_glu
y_true = y_true_z * std_glu + mu_glu

In [13]:
def convert_categorical(y):
    y_class = np.full((len(y)), "unknown", dtype='U10')
    y_hypo_mask = y <= 70
    y_hyper_mask = y >= 180
    y_normal_mask = (y > 70) & (y < 180)
    y_class[y_hypo_mask] = "Hypo"
    y_class[y_hyper_mask] = "Hyper"
    y_class[y_normal_mask] = "Normal"
    return y_class

In [14]:
y_pred_class = convert_categorical(y_pred)
y_true_class = convert_categorical(y_true)

In [15]:
le = LabelEncoder()
y_true_encoded = le.fit_transform(y_true_class)  # e.g. 'Hypo'=0, 'Normal'=1, 'Hyper'=2
y_pred_encoded = le.transform(y_pred_class)

In [16]:
# Full classification metrics
print(classification_report(y_true_encoded, y_pred_encoded, target_names=le.classes_))

# Confusion matrix
cm = confusion_matrix(y_true_encoded, y_pred_encoded)
print("Confusion Matrix:\n", cm)

              precision    recall  f1-score   support

       Hyper       0.46      0.00      0.01     15848
        Hypo       0.00      0.00      0.00      1155
      Normal       0.80      1.00      0.89     67760

    accuracy                           0.80     84763
   macro avg       0.42      0.33      0.30     84763
weighted avg       0.73      0.80      0.71     84763

Confusion Matrix:
 [[   53     0 15795]
 [    1     0  1154]
 [   60     0 67700]]


C:\Users\Mashal\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Mashal\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Mashal\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
# One-hot encode
y_true_bin = label_binarize(y_true_encoded, classes=[0,1,2])
y_pred_bin = label_binarize(y_pred_encoded, classes=[0,1,2])

# Macro-average AUC
auc_macro = roc_auc_score(y_true_bin, y_pred_bin, average='macro', multi_class='ovo')
print(f"Macro-average AUC-ROC: {auc_macro:.4f}")

Macro-average AUC-ROC: 0.5008
